In [3]:
import sys

sys.path.append("../..")
sys.path.append("/vol/paramonos2/projects/antoni/code/Personal/lipreader")
from lipreader.code.lightnining import ModelModule
import torch


def get_lightning_module(ckpt_path):
    modelmodule = ModelModule()
    modelmodule.model.load_state_dict(torch.load(ckpt_path, map_location=lambda storage, loc: storage))
    return modelmodule


ckpt_path = "../models/vsr_trlrs3_base.max400.pth"
model = get_lightning_module(ckpt_path)
model.eval()

ModuleNotFoundError: No module named 'datamodule'